#Install Package

In [ ]:
# !pip install openai --quiet

In [ ]:
# !pip install langchain

In [ ]:
# !pip install langchain_openai

In [ ]:
# !pip install azure-cognitiveservices-speech pydub 

# Set API Key

In [ ]:
from langchain_openai import ChatOpenAI
from openai import OpenAI

import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = "sk-proj-9XxUFpy63A4GwHsxaK1PT3BlbkFJmPCuVmTlFJ3bNJCgiOp2"

client = OpenAI()

# Propt

In [ ]:
information_extrac_prompt = """ Your job is to extract important information from USER text input
                                The important information you need to pull is:

                                INFORMATION_JSON:
                                {INFROMATION_JSON}

                                This is the previous question you asked:
                                {QUESTION}

                                USER: {USER_INPUT_DATA}

                                If in the USER message is available in INFORMATION.
                                Please reply this information in json format.

                                Do not delete existing data. Please return the original data as well. But if the information is on the same topic Please combine the information.

                                Please write information with USET input language."""

In [ ]:
# conversatoin_prompt = """ You are an investigative expert. Your job is to try and answer questions.
#                           with the user in order to collect information that is useful in further investigating the case Complete and complete

#                           INFORMATION_JSON:
#                           {INFORMATION_JSON}

#                           This is the previous question you asked:
#                           {QUESTION}

#                           Don't ask the same question again.

#                           If some any part has INFORMATION_JSON but is not complete. Please ask questoin more for gave the information from user, but if the INFORMATION_JSON is complete Please ask more the new next question.
#                           Try to ask one question at a time, and reply with a sympathetic message.

#                           If there is no information at all Start by asking your name first.

#                           Try asking in a new way, don't repeat the same pattern like this previous question -> {QUESTION}.

#                           Remember asking as if He was talking to a police officer. who is really a female person.

#                           Response with Thai language.
#                                     """

In [ ]:
conversatoin_prompt = """ You are an investigative expert. Your job is to try and answer questions.
                          with the user in order to collect information that is useful in further investigating complete.

                          INFORMATION_JSON:
                          {INFORMATION_JSON}

                          This is the previous question you asked:
                          {QUESTION}

                          Don't ask the same question again.

                          If some any part has INFORMATION_JSON but is not complete. Please ask questoin more for gave the information from user, but if the INFORMATION_JSON is complete Please ask more the new next question.
                          
                          Try to ask one question at a time.

                          If there is no information at all Start by asking your name first.

                          Remember asking as if He was talking to a police officer. who is really a female person.
                          
                          Please note the new question and avoid repeating the format of the previous question, which was -> {QUESTION}.

                          

                          

                          Response with Thai language.
                                    """

In [ ]:
# Please only show empathy or understanding or suggestion, and questions. Don't talk about anything else or say anything else.

In [ ]:
data_from_user = {
                  "ชื่อ" : "",
                  "เหตุการณ์ที่เกิดขึ้น": "",
                  "ชื่อคนร้าย": "",
                  "วัน/เวลาที่เกิดเหตุ": "",
                  "ประเภททรัพย์สินที่เสียไป": "",
                  "มูลค่าความเสียหายที่เกิดขึ้น": "",
                  "หมายเลขบัญชีของคนร้าย": "",
                  "ช่องทางที่คนร้ายติดต่อเข้ามา": "",
                  "คนร้ายติดต่อมาด้วยข้อความแบบใหน/ตัวอย่าง": "",
                  "มีช่องทางในการติดต่อคนร้ายใหม": "",
}

# Information Extraction

In [ ]:
from typing import Any, Dict
import ast
import json

def information_extract(user_input: str, question: str, data_from_user: Dict )-> Dict:
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "user", "content": information_extrac_prompt.replace("{USER_INPUT_DATA}",user_input).replace('{QUESTION}',question).replace('{INFROMATION_JSON}',str(data_from_user)) }
    ],
    temperature=0,
    response_format={ "type": "json_object" }
  )

  result_dict = ast.literal_eval(completion.choices[0].message.content)
  return result_dict


# Queston Generation


In [ ]:
def generate_question(question: str) -> str:
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "user", "content": conversatoin_prompt.replace('{INFORMATION_JSON}',str(data_from_user)).replace('{QUESTION}',question)}],
    temperature=0.7,
  )
  return completion.choices[0].message.content

# Check information in Json


In [ ]:
def check_information(data_dict: Dict) -> bool:
  check = True
  for valuse in data_dict.values():
    if valuse == "" :
      check = False
  return check

# Text to Speech by Azure

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk

os.environ['SPEECH_KEY'] = "d0289464af034bdca915c7f2af4bac75"
os.environ['SPEECH_REGION'] = "eastus"

# This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
text2speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
text2speech_audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# The neural multilingual voice can speak different languages based on the input text.
text2speech_config.speech_synthesis_voice_name = 'th-TH-PremwadeeNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=text2speech_config, audio_config=text2speech_audio_config)

# Function to convert text to speech with slow rate for Thai language
def text_to_speech_azure(text, rate="-15%"):
    ssml_text = f"""
    <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="th-TH">
        <voice name="{text2speech_config.speech_synthesis_voice_name}">
            <prosody rate="{rate}">
                {text}
            </prosody>
        </voice>
    </speak>
    """

    speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print(f"Speech synthesized for text [{text}] with rate [{rate}]")
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print(f"Speech synthesis canceled: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print(f"Error details: {cancellation_details.error_details}")
                print("Did you set the speech resource key and region values?")

text_to_speech_azure("สวัสดีค่ะ ดิฉันเป็นเจ้าหน้าที่ตำรวจที่กำลังดูแลคดีนี้อยู่ ขอทราบชื่อของคุณเพื่อที่เราจะได้เริ่มต้นทำการสอบสวนได้ค่ะ")

# Text to Speech by Botnoi

In [ ]:
import requests
from pydub import AudioSegment
import json
import simpleaudio as sa

def text_to_speech_botnoi(text):
    #requests botnoi API
    url = "https://api-voice.botnoi.ai/openapi/v1/generate_audio"
    payload = {"text":text, "speaker":"14", "volume":1, "speed":1, "type_media":"m4a", "save_file": "true", "language": "th"}
    headers = {
    'Botnoi-Token': 'c24wZ3FDNW1pUFZXWmlaeHBRcFhNcnFrVzlBMzU2MTg5NA==',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, json=payload)

    url = response.content
    
    url_str = url.decode('utf-8')

    # Parse the JSON string
    data = json.loads(url_str)
    print(data)
    # Extract the audio_url
    audio_url = data['audio_url']
    print(audio_url)
    # Download the file
    response = requests.get(audio_url)
    m4a_filename = 'audio_file.m4a'
    with open(m4a_filename, 'wb') as f:
        f.write(response.content)

    # Convert the file to WAV format
    wav_filename = 'audio_file.wav'
    audio = AudioSegment.from_file(m4a_filename, format='m4a')
    audio.export(wav_filename, format='wav')

    wave_obj = sa.WaveObject.from_wave_file(wav_filename)
    # Play the WAV file
    play_obj = wave_obj.play()
    play_obj.wait_done()  # Wait until sound has finished playing



# Speech To Text by Azure

In [ ]:
# This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
speech2text_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
speech2text_config.speech_recognition_language="th-TH"

speech2text_audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech2text_config, audio_config=speech2text_audio_config)

def speech_to_text():
    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")


# Combine all of this together.

In [ ]:
from pprint import pprint

question = ""

while True:
  question = generate_question(question)
  print(question)
  text_to_speech_azure(question)
  # text_to_speech_botnoi(question)
  user_input = speech_to_text()
  # user_input = input()
  if user_input == "q":
    break
  data_from_user = information_extract(user_input,question,data_from_user)
  print('\n##################################\n')
  pprint(data_from_user)
  print('\n##################################\n')
  verify = check_information(data_from_user)
  if verify:
    break

In [ ]:
pprint(data_from_user)